In [1]:
# 这个进行ECG_unet的训练
!gpustat

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for
       usage information.

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for
       usage information.

ubuntu                      Thu Feb  1 19:03:26 2024  510.47.03
[0] NVIDIA GeForce RTX 3090 | 36'C,   0 % |   311 / 24576 MB |
[1] NVIDIA GeForce RTX 3090 | 37'C,   0 % |   311 / 24576 MB |
[2] NVIDIA GeForce RTX 3090 | 45'C,   0 % |   311 / 24576 MB |
[3] NVIDIA GeForce RTX 3090 | 35'C,   0 % |   311 / 24576 MB |
[4] NVIDIA GeForce RTX 3090 | 38'C,   0 % |   311 / 24576 MB |
[5] NVIDIA GeForce RTX 3090 | 57'C,  13 % | 14328 / 24576 MB | robotics(14017M)
[6] NVIDIA GeForce RTX 3090 | 52'C,  14 % | 14328 / 24576 MB | robotics(14017M)
[7] NVIDIA GeForce RTX 3090 | 56'C,  14 % | 14328 / 24576 MB | robotics(14017M)


In [1]:
# 使用显卡定义，库函数的引用
import os

os.environ["CUDA_VISIBLE_DEVICES"] ='2'
device='cuda:0'
from torch.autograd import Variable
import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt

import torch.backends.cudnn as cudnn
import sys
from datasets import train_loader
from torch.optim import lr_scheduler
import torch.nn.init as init
from models.DG_models import generate_model
from torch.nn import functional as F


In [2]:
# utils

def weights_init(init_type='gaussian'):
    def init_fun(m):
        classname = m.__class__.__name__
        if (classname.find('Conv') == 0 or classname.find('Linear') == 0) and hasattr(m, 'weight'):
            # print m.__class__.__name__
            if init_type == 'gaussian':
                init.normal_(m.weight.data, 0.0, 0.02)
            elif init_type == 'xavier':
                init.xavier_normal_(m.weight.data, gain=math.sqrt(2))
            elif init_type == 'kaiming':
                init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
            elif init_type == 'orthogonal':
                init.orthogonal_(m.weight.data, gain=math.sqrt(2))
            elif init_type == 'default':
                pass
            else:
                assert 0, "Unsupported initialization: {}".format(init_type)
            if hasattr(m, 'bias') and m.bias is not None:
                init.constant_(m.bias.data, 0.0)

    return init_fun


    
    
def get_model_list(dirname, key):
    if os.path.exists(dirname) is False:
        return None
    gen_models = [os.path.join(dirname, f) for f in os.listdir(dirname) if
                  os.path.isfile(os.path.join(dirname, f)) and key in f and ".pt" in f]
    if gen_models is None:
        return None
    gen_models.sort()
    last_model_name = gen_models[-1]
    return last_model_name
    
    
    
def get_scheduler(optimizer, iterations=-1):
    scheduler = lr_scheduler.StepLR(optimizer, step_size=100000,
                                        gamma=0.5, last_epoch=iterations)
    return scheduler



class Timer:
    def __init__(self, msg):
        self.msg = msg
        self.start_time = None

    def __enter__(self):
        self.start_time = time.time()

    def __exit__(self, exc_type, exc_value, exc_tb):
        print(self.msg % (time.time() - self.start_time))



In [8]:
# 网络定义，包括gen_a,gen_b,dis_a,dis_b

class AdaptiveInstanceNorm2d(nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super(AdaptiveInstanceNorm2d, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        # weight and bias are dynamically assigned
        self.weight = None
        self.bias = None
        # just dummy buffers, not used
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))

    def forward(self, x):
        assert self.weight is not None and self.bias is not None, "Please assign weight and bias before calling AdaIN!"
        b, c = x.size(0), x.size(1)
        running_mean = self.running_mean.repeat(b)
        running_var = self.running_var.repeat(b)

        # Apply instance norm
        x_reshaped = x.contiguous().view(1, b * c, *x.size()[2:])
        out = F.batch_norm(
            x_reshaped, running_mean, running_var, self.weight, self.bias,
            True, self.momentum, self.eps)

        return out.view(b, c, *x.size()[2:])

    def __repr__(self):
        return self.__class__.__name__ + '(' + str(self.num_features) + ')'

    
class Decoder(nn.Module):
    def __init__(self,):
        super(Decoder, self).__init__()

        # dec
        self.conv1=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn1=AdaptiveInstanceNorm2d(64)
        self.ac1=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv2=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn2=AdaptiveInstanceNorm2d(64)
        self.ac2=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv3=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn3=AdaptiveInstanceNorm2d(64)
        self.ac3=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        
        
        self.conv4=nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.bn4=AdaptiveInstanceNorm2d(32)
        self.ac4=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv5=nn.Conv1d(32, 32, kernel_size=3, padding=1)
        self.bn5=AdaptiveInstanceNorm2d(32)
        self.ac5=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv6=nn.Conv1d(32, 16, kernel_size=3, padding=1)
        self.bn6=AdaptiveInstanceNorm2d(16)
        self.ac6=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv7=nn.Conv1d(16, 16, kernel_size=3, padding=1)
        self.bn7=AdaptiveInstanceNorm2d(16)
        self.ac7=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv8=nn.Conv1d(16, 8, kernel_size=3, padding=1)
        self.bn8=AdaptiveInstanceNorm2d(8)
        self.ac8=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv9=nn.Conv1d(8, 8, kernel_size=3, padding=1)
        self.ac9=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv10=nn.Conv1d(8, 1, kernel_size=1)
        
    def forward(self, x):
        
        x=self.ac1(x+self.bn1(self.conv1(x)))
        x=self.ac2(x+self.bn2(self.conv2(x)))
        x=self.ac3(x+self.bn3(self.conv3(x)))
        x=self.ac4(self.bn4(self.conv4(x)))
        x=self.ac5(self.bn5(self.conv5(x)))
        x=self.ac6(self.bn6(self.conv6(x)))
        x=self.ac7(self.bn7(self.conv7(x)))
        x=self.ac8(self.bn8(self.conv8(x)))
        x=self.conv10(self.ac9(self.conv9(x)))
        
        return x


class ECG_gen(nn.Module):  
    def __init__(self,):  
        super(ECG_gen, self).__init__()  
        
        # cont_enc
        
        self.conv1=nn.Conv1d(1, 8, kernel_size=3, padding=1)
        self.bn1=nn.InstanceNorm1d(8)
        self.ac1=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv2=nn.Conv1d(8, 8, kernel_size=3, padding=1)
        self.bn2=nn.InstanceNorm1d(8)
        self.ac2=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv3=nn.Conv1d(8, 16, kernel_size=3, padding=1)
        self.bn3=nn.InstanceNorm1d(16)
        self.ac3=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv4=nn.Conv1d(16, 16, kernel_size=3, padding=1)
        self.bn4=nn.InstanceNorm1d(16)
        self.ac4=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv5=nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.bn5=nn.InstanceNorm1d(32)
        self.ac5=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv6=nn.Conv1d(32, 32, kernel_size=3, padding=1)
        self.bn6=nn.InstanceNorm1d(32)
        self.ac6=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv7=nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.bn7=nn.InstanceNorm1d(64)
        self.ac7=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv8=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn8=nn.InstanceNorm1d(64)
        self.ac8=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        
        
        self.conv9=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn9=nn.InstanceNorm1d(64)
        self.ac9=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv10=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn10=nn.InstanceNorm1d(64)
        self.ac10=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv11=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn11=nn.InstanceNorm1d(64)
        self.ac11=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        # style_enc
        
        self.conv_sty_1=nn.Conv1d(1, 8, kernel_size=3, padding=1)
        self.ac_sty_1=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv_sty_2=nn.Conv1d(8, 16, kernel_size=3, padding=1)
        self.ac_sty_2=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv_sty_3=nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.ac_sty_3=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv_sty_4=nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.ac_sty_4=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv_sty_5=nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.ac_sty_5=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv_sty_6=nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.ac_sty_6=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.pool_sty=nn.AdaptiveAvgPool1d(1)
        self.fc_sty=nn.Conv1d(256, 8, kernel_size=1, padding=0)
        # bs,8,1
        
        self.dec=Decoder()
        
        self.mlp_1=nn.Linear(8, 256, bias=True)
        self.mlp_2=nn.LeakyReLU(0.01, inplace=True)
        self.mlp_3=nn.Linear(256, 256, bias=True)
        self.mlp_4=nn.LeakyReLU(0.01, inplace=True)
        self.mlp_5=nn.Linear(256, self.get_num_adain_params(self.dec), bias=True)
        
        
    def encode(self, x):  # bs,1000,1  x1是特征，x2是风格
        
        x=x.transpose(-1,-2) # bs,1,1000
        
        xc=self.conv1(x)
        xc=self.bn1(xc)
        xc=self.ac1(xc)
        xc=self.ac2(self.bn2(self.conv2(xc)))
        xc=self.ac2(self.bn2(self.conv2(xc)))
        xc=self.ac3(self.bn3(self.conv3(xc)))
        xc=self.ac4(self.bn4(self.conv4(xc)))
        xc=self.ac5(self.bn5(self.conv5(xc)))
        xc=self.ac6(self.bn6(self.conv6(xc)))
        xc=self.ac7(self.bn7(self.conv7(xc)))
        xc=self.ac8(self.bn8(self.conv8(xc)))
        xc=self.ac9(xc+self.bn9(self.conv9(xc)))
        xc=self.ac10(xc+self.bn10(self.conv10(xc)))
        content=self.ac11(xc+self.bn11(self.conv11(xc)))   # bs,64,1000

        
        
        xs=self.conv_sty_1(x)
        xs=self.ac_sty_1(xs)
        xs=self.conv_sty_2(xs)
        xs=self.ac_sty_2(xs)
        xs=self.conv_sty_3(xs)
        xs=self.ac_sty_3(xs)
        xs=self.conv_sty_4(xs)
        xs=self.ac_sty_4(xs)
        xs=self.conv_sty_5(xs)
        xs=self.ac_sty_5(xs)
        xs=self.conv_sty_6(xs)
        xs=self.ac_sty_6(xs)
        xs=self.pool_sty(xs)
        style=self.fc_sty(xs)
        
        return content, style
    
    def decode(self,content, style):  # x1逆向操作，x2应用到自适应归一化层
        
        adain_params = self.mlp_1(style.view(style.size(0), -1))
        adain_params = self.mlp_2(adain_params)
        adain_params = self.mlp_3(adain_params)
        adain_params = self.mlp_4(adain_params)
        adain_params = self.mlp_5(adain_params)
        self.assign_adain_params(adain_params, self.dec)
        x=self.dec(content)
        x=x.transpose(-1,-2)
        
        return x
        
        
    def assign_adain_params(self, adain_params, model):
        for m in model.modules():
            if m.__class__.__name__ == "AdaptiveInstanceNorm2d":
                mean = adain_params[:, :m.num_features]
                std = adain_params[:, m.num_features:2*m.num_features]
                m.bias = mean.contiguous().view(-1)
                m.weight = std.contiguous().view(-1)
                if adain_params.size(1) > 2*m.num_features:
                    adain_params = adain_params[:, 2*m.num_features:]
        
        
    def get_num_adain_params(self,model):
        num_adain_params = 0
        for m in model.modules():
            if m.__class__.__name__ == "AdaptiveInstanceNorm2d":
                num_adain_params += 2*m.num_features
        return num_adain_params
        
    def forward(self, images):
        # reconstruct an image
        content, style_fake = self.encode(images)
        images_recon = self.decode(content, style_fake)
        return images_recon
    
    
    
class ECG_dis(nn.Module):  
    def __init__(self,):  
        super(ECG_dis, self).__init__()  
        self.lead1_net=generate_model(base_model='resnet18',input_channels=1, num_classes=1,DG_method=None,domain_classes=6,distill=False)
        
    
    def forward(self, x):
        outputs = self.lead1_net(x)
        return outputs

    
    def calc_dis_loss(self, input_fake, input_real):
        # calculate the loss to train D
        outs0 = self.forward(input_fake)
        outs1 = self.forward(input_real)
        loss = 0

        for it, (out0, out1) in enumerate(zip(outs0, outs1)):
            loss += torch.mean((out0 - 0)**2) + torch.mean((out1 - 1)**2)
        return loss

    def calc_gen_loss(self, input_fake):
        # calculate the loss to train G
        outs0 = self.forward(input_fake)
        loss = 0
        for it, (out0) in enumerate(outs0):
            loss += torch.mean((out0 - 1)**2) # LSGAN
        return loss
    
    
    
    
    
    
    
class ECG_UNIT_Trainer(nn.Module):
    def __init__(self):
        super(ECG_UNIT_Trainer, self).__init__()
        self.count=0
        lr=0.0001
        self.gen_a=ECG_gen().to(device)
        self.gen_b=ECG_gen().to(device)
        self.dis_a=ECG_dis().to(device)
        self.dis_b=ECG_dis().to(device)
        self.style_dim=8
        
        self.gen_loss=0
        self.dis_loss=0
        self.show_xba=None
        self.show_xab=None
        self.show_xa=None
        self.show_xb=None
        
        
        # fix the noise used in sampling
        display_size = 16
        self.s_a = torch.randn(display_size, self.style_dim).to(device)
        self.s_b = torch.randn(display_size, self.style_dim).to(device)
        
        dis_params = list(self.dis_a.parameters()) + list(self.dis_b.parameters())
        gen_params = list(self.gen_a.parameters()) + list(self.gen_b.parameters())
        self.dis_opt = torch.optim.Adam([p for p in dis_params if p.requires_grad],
                                        lr=lr, betas=(0.5, 0.999), weight_decay=0.0001)
        self.gen_opt = torch.optim.Adam([p for p in gen_params if p.requires_grad],
                                        lr=lr, betas=(0.5, 0.999), weight_decay=0.0001)
        self.dis_scheduler = get_scheduler(self.dis_opt)
        self.gen_scheduler = get_scheduler(self.gen_opt)
        
        self.apply(weights_init('kaiming'))
        self.gen_b.apply(weights_init('kaiming'))
        self.dis_a.apply(weights_init('gaussian'))
        self.dis_b.apply(weights_init('gaussian'))

    def recon_criterion(self, input, target):
        return torch.mean(torch.abs(input - target))
        
    def forward(self, x_a, x_b):
        self.eval()
        s_a = Variable(self.s_a)
        s_b = Variable(self.s_b)
        c_a, s_a_fake = self.gen_a.encode(x_a)
        c_b, s_b_fake = self.gen_b.encode(x_b)
        x_ba = self.gen_a.decode(c_b, s_a)
        x_ab = self.gen_b.decode(c_a, s_b)
        self.train()
        return x_ab, x_ba
    
    def gen_update(self, x_a, x_b):
        self.gen_opt.zero_grad()
        s_a = Variable(torch.randn(x_a.size(0), self.style_dim).to(device))
        s_b = Variable(torch.randn(x_b.size(0), self.style_dim).to(device))
        # encode
        c_a, s_a_prime = self.gen_a.encode(x_a)
        c_b, s_b_prime = self.gen_b.encode(x_b)
        # decode (within domain)
        x_a_recon = self.gen_a.decode(c_a, s_a_prime)
        x_b_recon = self.gen_b.decode(c_b, s_b_prime)
        # decode (cross domain)
        x_ba = self.gen_a.decode(c_b, s_a)
        x_ab = self.gen_b.decode(c_a, s_b)
        # encode again
        c_b_recon, s_a_recon = self.gen_a.encode(x_ba)
        c_a_recon, s_b_recon = self.gen_b.encode(x_ab)

        # reconstruction loss
        self.loss_gen_recon_x_a = self.recon_criterion(x_a_recon, x_a)
        self.loss_gen_recon_x_b = self.recon_criterion(x_b_recon, x_b)
        self.loss_gen_recon_s_a = self.recon_criterion(s_a_recon, s_a)
        self.loss_gen_recon_s_b = self.recon_criterion(s_b_recon, s_b)
        self.loss_gen_recon_c_a = self.recon_criterion(c_a_recon, c_a)
        self.loss_gen_recon_c_b = self.recon_criterion(c_b_recon, c_b)
        # GAN loss
        self.loss_gen_adv_a = self.dis_a.calc_gen_loss(x_ba)
        self.loss_gen_adv_b = self.dis_b.calc_gen_loss(x_ab)
        # total loss
        self.loss_gen_total = 1 * self.loss_gen_adv_a + \
                              1 * self.loss_gen_adv_b + \
                              10 * self.loss_gen_recon_x_a + \
                              1 * self.loss_gen_recon_s_a + \
                              1 * self.loss_gen_recon_c_a + \
                              10 * self.loss_gen_recon_x_b + \
                              1 * self.loss_gen_recon_s_b + \
                              1 * self.loss_gen_recon_c_b

        self.loss_gen_total.backward()
        
        if ((self.gen_loss)<(0.1*self.dis_loss))and(self.gen_loss!=0):
            pass
        else:
            self.gen_opt.step()
        self.gen_loss=self.loss_gen_total.item()
        
        
        
    def dis_update(self, x_a, x_b):
        self.dis_opt.zero_grad()
        s_a = Variable(torch.randn(x_a.size(0), self.style_dim).to(device))
        s_b = Variable(torch.randn(x_b.size(0), self.style_dim).to(device))
        # encode
        c_a, _ = self.gen_a.encode(x_a)
        c_b, _ = self.gen_b.encode(x_b)
        # decode (cross domain)
        x_ba = self.gen_a.decode(c_b, s_a)
        x_ab = self.gen_b.decode(c_a, s_b)
        # D loss
        self.loss_dis_a = self.dis_a.calc_dis_loss(x_ba.detach(), x_a)
        self.loss_dis_b = self.dis_b.calc_dis_loss(x_ab.detach(), x_b)
        self.loss_dis_total = 1 * self.loss_dis_a + 1 * self.loss_dis_b
        self.loss_dis_total.backward()
        
        if ((self.gen_loss)>(10*self.dis_loss))and(self.dis_loss!=0):
            pass
        else:
            self.dis_opt.step()
#             if (self.count+1)%1000==0:
        self.dis_loss=self.loss_dis_total.item()
        self.show_xa=x_a.detach().cpu().numpy()
        self.show_xb=x_b.detach().cpu().numpy()
        self.show_xab=x_ab.detach().cpu().numpy()
        self.show_xba=x_ba.detach().cpu().numpy()
        
    def show_loss(self,):
        print('gen_loss=',self.gen_loss)
        print('dis_loss=',self.dis_loss)
        
        with open('2024log0201_ECG_MBDG.txt', 'a') as file:  
            file.write('______________'+str(self.count)+'_____________________________ \n')
            file.write('gen_loss \n')
            file.write(str(self.gen_loss)+'\n')
            file.write('dis_loss \n')
            file.write(str(self.dis_loss)+'\n')
            
        plt.plot(self.show_xb[0,:])
        plt.plot(self.show_xab[0,:])
        plt.savefig('ECG_MBDG_figs/b_'+str(self.count)+'.png')
        plt.show()
        plt.plot(self.show_xa[0,:])
        plt.plot(self.show_xba[0,:])
        plt.savefig('ECG_MBDG_figs/a_'+str(self.count)+'.png')
        plt.show()
        

        print('__________________________')
        

    def update_learning_rate(self):
        if self.dis_scheduler is not None:
            self.dis_scheduler.step()
            self.count+=1
        if self.gen_scheduler is not None:
            self.gen_scheduler.step()

    def resume(self, checkpoint_dir):
        # Load generators
        last_model_name = get_model_list(checkpoint_dir, "gen")
        state_dict = torch.load(last_model_name)
        self.gen_a.load_state_dict(state_dict['a'])
        self.gen_b.load_state_dict(state_dict['b'])
        iterations = int(last_model_name[-11:-3])
        # Load discriminators
        last_model_name = get_model_list(checkpoint_dir, "dis")
        state_dict = torch.load(last_model_name)
        self.dis_a.load_state_dict(state_dict['a'])
        self.dis_b.load_state_dict(state_dict['b'])
        # Load optimizers
        state_dict = torch.load(os.path.join(checkpoint_dir, 'optimizer.pt'))
        self.dis_opt.load_state_dict(state_dict['dis'])
        self.gen_opt.load_state_dict(state_dict['gen'])
        # Reinitilize schedulers
        self.dis_scheduler = get_scheduler(self.dis_opt,  iterations)
        self.gen_scheduler = get_scheduler(self.gen_opt,  iterations)
        print('Resume from iteration %d' % iterations)
        return iterations

    def save(self, snapshot_dir, iterations):
        # Save generators, discriminators, and optimizers
        gen_name = os.path.join(snapshot_dir, 'gen_%08d.pt' % (iterations + 1))
        dis_name = os.path.join(snapshot_dir, 'dis_%08d.pt' % (iterations + 1))
        opt_name = os.path.join(snapshot_dir, 'optimizer.pt')
        torch.save({'a': self.gen_a.state_dict(), 'b': self.gen_b.state_dict()}, gen_name)
        torch.save({'a': self.dis_a.state_dict(), 'b': self.dis_b.state_dict()}, dis_name)
        torch.save({'gen': self.gen_opt.state_dict(), 'dis': self.dis_opt.state_dict()}, opt_name)
        
        


In [4]:
# 数据集加载

train_data_list=['L','B','C','A','N','H']

train_datasloader_instance=train_loader(dataname=train_data_list,train_val_test_split=[0.8,0.2,1],bs=1,num_workers=2,smote=False,maple=False,class_num=3)
train_loader_a=train_datasloader_instance.loader()

train_datasloader_instance=train_loader(dataname=train_data_list,train_val_test_split=[0.8,0.2,1],bs=1,num_workers=2,smote=False,maple=False,class_num=3)
train_loader_b=train_datasloader_instance.loader()

train data L
train data size : 16824
train label class :  [13511.  1158.  2155. 16824.     0.     0.     0.     0.     0.]
train data B
train data size : 5501
train label class :  [ 742.  859. 3900.    0. 5501.    0.    0.    0.    0.]
train data C
train data size : 24194
train label class :  [ 6719.  2039. 15436.     0.     0. 24194.     0.     0.     0.]
train data A
train data size : 8274
train label class :  [1393.  480. 6401.    0.    0.    0. 8274.    0.    0.]
train data N
train data size : 8516
train label class :  [1462. 1769. 5285.    0.    0.    0.    0. 8516.    0.]
train data H
train data size : 20616
train label class :  [11152.   557.  8907.     0.     0.     0.     0.     0. 20616.]
total train data shape =  (83925, 1000, 1)
total train label shape =  (83925, 9)
total train data class =  [34979.  6862. 42084. 16824.  5501. 24194.  8274.  8516. 20616.]
train data L
train data size : 16824
train label class :  [13511.  1158.  2155. 16824.     0.     0.     0.     0.     0

In [9]:
# 开始训练流程，定期保存网络和优化器

cudnn.benchmark = True

max_iter = 4500000

trainer = ECG_UNIT_Trainer()
trainer.to(device)


# 已经shuffle过了所以顺序没问题

resume=False
checkpoint_directory='save_model_states/'

# Start training
iterations = 0
# = trainer.resume(checkpoint_directory) if resume else 0
for epoch in range(50):
    for it, (images_a_, images_b_) in enumerate(zip(train_loader_a, train_loader_b)):
        trainer.update_learning_rate()
        images_a, images_b = images_a_[0].float().to(device).detach(), images_b_[0].float().to(device).detach()

    #     with Timer("Elapsed time in update: %f"):
    #         # Main training code
        trainer.dis_update(images_a, images_b)
        trainer.gen_update(images_a, images_b)


        if (iterations + 1) % 1000 == 0:
            print('iterations='+str(iterations))
            trainer.show_loss()
            print('__________________________________')

        # Save network weights
        if (iterations + 1) % 20000 == 0:
            trainer.save(checkpoint_directory, iterations)

        iterations += 1
        if iterations >= max_iter:
            sys.exit('Finish training')


KeyboardInterrupt: 